In [20]:
import torch
import torchvision
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import pickle
from copy import deepcopy
from scipy import stats
    
from torchsummary import summary
import math
import random
import pandas as pd

from sklearn.model_selection import train_test_split

In [21]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)



class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)


In [22]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))        


In [23]:
print("Program To train a base neuron for Pima")

print("Torch cuda ",torch.cuda.is_available())


device = get_default_device()
print("device ",device)


Program To train a base neuron for Pima
Torch cuda  False
device  cpu


In [24]:
df=pd.read_csv("pima-indians-diabetes.data.csv",header=None)
print(df.head())

   0    1   2   3    4     5      6   7  8
0  6  148  72  35    0  33.6  0.627  50  1
1  1   85  66  29    0  26.6  0.351  31  0
2  8  183  64   0    0  23.3  0.672  32  1
3  1   89  66  23   94  28.1  0.167  21  0
4  0  137  40  35  168  43.1  2.288  33  1


In [25]:
X=df.loc[:,:7].values
y=df.loc[:,8].values

In [26]:
X.shape

(768, 8)

In [27]:
y.shape

(768,)

In [28]:

X_train,X_test,y_train,y_test = train_test_split(X,y , test_size =0.2,random_state=0)



In [29]:
# Creating Tensors
X_train=torch.FloatTensor(X_train)
X_test=torch.FloatTensor(X_test)
y_train=torch.LongTensor(y_train)
y_test=torch.LongTensor(y_test)

### Below just to check if weights are working the way we think they are

In [30]:
test_weight1=torch.FloatTensor(np.array([ 0.0445, -0.1208,  0.2645, -0.0978,  0.0318,  0.2220,  0.2510, -0.2588]))
bias1=torch.tensor(-0.0446)
test_weight2=torch.FloatTensor(np.array([-0.2218,  0.1974, -0.3320, -0.1531,  0.0914, -0.1960,  0.1685,  0.0053]))
bias2=torch.tensor(-0.0413)

In [31]:
print(torch.matmul(X_train[0],test_weight1)+bias1)
print(torch.matmul(X_train[0],test_weight2)+bias2)

tensor(-2.0386)
tensor(2.7002)


In [32]:
print(torch.matmul(X_train[1],test_weight1)+bias1)
print(torch.matmul(X_train[1],test_weight2)+bias2)

tensor(2.7144)
tensor(-10.5580)


### Above matches with the other notebok output

In [33]:
initial_weights=torch.FloatTensor([[ 0.0435, -0.2281,  0.2225, -0.1205, -0.1896,  0.2002,  0.2506, -0.2820],
        [-0.2208,  0.3048, -0.2900, -0.1304,  0.3128, -0.1742,  0.1689,  0.0285]])


outs=torch.matmul(initial_weights,X_train.T)
outs=outs.T
initial_acc=accuracy(outs, y_train)

print("Initial acc is ",initial_acc)

Initial acc is  tensor(0.3713)


In [34]:
random.choices([1,-1],weights=[0.01,0.99])

[-1]

In [35]:
num_weights=16
counter=3000

initial_probabs=[[0.5,0.5] for i in range(num_weights)]
# print("for each wts initial probabs=",initial_probabs)
delta=0.01
options=[1,-1]
# changes=[]
# for i in range(num_weights):
#     this_wt_change=torch.tensor(random.choices(options,weights=initial_probabs[i],k=1))
#     this_wt_change=this_wt_change*delta
#     changes.append(this_wt_change)
# changes=torch.FloatTensor(changes).reshape(initial_weights.shape)    
# print(changes)
# new_weights=initial_weights+changes

new_weights=deepcopy(initial_weights)

for count in range(counter):
    outs=torch.matmul(new_weights,X_train.T)
    outs=outs.T
    acc=accuracy(outs, y_train)
    changes=[]
    for i in range(num_weights):        
        this_wt_change=torch.tensor(random.choices(options,weights=initial_probabs[i],k=1))
        this_wt_change=this_wt_change*delta
        changes.append(this_wt_change)
    changes=torch.FloatTensor(changes).reshape(initial_weights.shape)    
    new_weights=new_weights+changes
    if count%100==0:
        print(count,"Accuracy is ",acc)        
#         print(new_weights)
#     break
    
    
print(new_weights)

0 Accuracy is  tensor(0.3713)
100 Accuracy is  tensor(0.3762)
200 Accuracy is  tensor(0.3893)
300 Accuracy is  tensor(0.4805)
400 Accuracy is  tensor(0.5570)
500 Accuracy is  tensor(0.5912)
600 Accuracy is  tensor(0.5945)
700 Accuracy is  tensor(0.5440)
800 Accuracy is  tensor(0.5489)
900 Accuracy is  tensor(0.5114)
1000 Accuracy is  tensor(0.5033)
1100 Accuracy is  tensor(0.4870)
1200 Accuracy is  tensor(0.4984)
1300 Accuracy is  tensor(0.5130)
1400 Accuracy is  tensor(0.5179)
1500 Accuracy is  tensor(0.5749)
1600 Accuracy is  tensor(0.6042)
1700 Accuracy is  tensor(0.6254)
1800 Accuracy is  tensor(0.6205)
1900 Accuracy is  tensor(0.5879)
2000 Accuracy is  tensor(0.5847)
2100 Accuracy is  tensor(0.5928)
2200 Accuracy is  tensor(0.6336)
2300 Accuracy is  tensor(0.6270)
2400 Accuracy is  tensor(0.6482)
2500 Accuracy is  tensor(0.6368)
2600 Accuracy is  tensor(0.6450)
2700 Accuracy is  tensor(0.6401)
2800 Accuracy is  tensor(0.6433)
2900 Accuracy is  tensor(0.6433)
tensor([[ 0.0235,  0.7

In [36]:
def probab_change(epsilon_change_probas,probab_pairs):
    for i in range(len(probab_pairs)):
        probab_pairs[i][0]=probab_pairs[i][0]+epsilon_change_probas[0]
        probab_pairs[i][1]=probab_pairs[i][1]+epsilon_change_probas[1]
    return probab_pairs

In [37]:
def mod_weights_by_proba(options,old_weights,change_probabs,delta):
#     print("delta=",delta)
    changes=[]
    for i in range(torch.numel(old_weights)):        
        this_wt_change=torch.tensor(random.choices(options,weights=pos_change_probabs[i],k=1))
        this_wt_change=this_wt_change*delta
        changes.append(this_wt_change)
    changes=torch.FloatTensor(changes).reshape(old_weights.shape)    
    change_weights=deepcopy(old_weights)+changes    
    return change_weights

### Want to generate three sets of probabs

In [41]:
num_weights=16
counter=16000
epsilon_change_proba=0.001
initial_probabs=[[0.5,0.5] for i in range(num_weights)]
# print("for each wts initial probabs=",initial_probabs)
delta=0.1
options=[1,-1]


new_weights=deepcopy(initial_weights)
best_so_far=-1
best_weights_so_far=None
for count in range(counter):
#     print("Epoch:",count)
#     print("let us generate the three sets of probabilities")
    epsilon_change_probas=[+epsilon_change_proba,-epsilon_change_proba]
    pos_change_probabs=probab_change(epsilon_change_probas,deepcopy(initial_probabs))
#     print(new_weights)
    pos_change_weights=mod_weights_by_proba(options,new_weights,pos_change_probabs,delta)
#     print("pos change weights = ",pos_change_weights)
    outs=torch.matmul(pos_change_weights,X_train.T)
    outs=outs.T
    acc_pos=accuracy(outs, y_train)
    
    
    
    epsilon_change_probas=[-epsilon_change_proba,+epsilon_change_proba]    
    neg_change_probabs=probab_change(epsilon_change_probas,deepcopy(initial_probabs)) 
    neg_change_weights=mod_weights_by_proba(options,new_weights,neg_change_probabs,delta)
#     print("neg change weights = ",neg_change_weights)
    outs=torch.matmul(neg_change_weights,X_train.T)
    outs=outs.T
    acc_neg=accuracy(outs, y_train)
    
    
    no_change_probabs=deepcopy(initial_probabs)
    no_change_weights=mod_weights_by_proba(options,new_weights,no_change_probabs,delta)
#     print("no change weights",no_change_weights)
    outs=torch.matmul(no_change_weights,X_train.T)
    outs=outs.T
    acc_none=accuracy(outs, y_train)
    
#     print("pos\t0\tneg")
#     print(acc_pos,acc_none,acc_neg)
    best=-1
    if acc_pos>=acc_none and acc_pos>=acc_neg:
        best=acc_pos
        initial_probabs=deepcopy(pos_change_probabs)
        new_weights=deepcopy(pos_change_weights)
    elif acc_neg>=acc_none and acc_neg>=acc_pos:
        best=acc_neg
        initial_probabs=deepcopy(neg_change_probabs)
        new_weights=deepcopy(neg_change_weights)        
    else:
        best=acc_none
        initial_probabs=deepcopy(no_change_probabs)
        new_weights=deepcopy(no_change_weights)        
    if best_so_far<best:
        best_so_far=best
        best_weights_so_far=deepcopy(new_weights)
#     elif best_so_far>best:
#         new_weights=deepcopy(best_weights_so_far)
    if count%500==0:
        print(delta,count,best)
#         delta=delta/2
#         epsilon_change_proba=epsilon_change_proba/2

    
print("best ever = ",best_so_far)
    
    
    
    
    
    
    


0.1 0 tensor(0.4495)
0.1 500 tensor(0.6873)
0.1 1000 tensor(0.7068)
0.1 1500 tensor(0.7036)
0.1 2000 tensor(0.7085)
0.1 2500 tensor(0.7036)
0.1 3000 tensor(0.6971)
0.1 3500 tensor(0.7068)
0.1 4000 tensor(0.7101)
0.1 4500 tensor(0.7036)
0.1 5000 tensor(0.7068)
0.1 5500 tensor(0.7085)
0.1 6000 tensor(0.7085)
0.1 6500 tensor(0.7085)
0.1 7000 tensor(0.7068)
0.1 7500 tensor(0.7068)
0.1 8000 tensor(0.7068)
0.1 8500 tensor(0.7068)
0.1 9000 tensor(0.7068)
0.1 9500 tensor(0.7068)
0.1 10000 tensor(0.7068)
0.1 10500 tensor(0.7068)
0.1 11000 tensor(0.7068)
0.1 11500 tensor(0.7068)
0.1 12000 tensor(0.7068)
0.1 12500 tensor(0.7068)
0.1 13000 tensor(0.7068)
0.1 13500 tensor(0.7068)
0.1 14000 tensor(0.7068)
0.1 14500 tensor(0.7068)
0.1 15000 tensor(0.7068)
0.1 15500 tensor(0.7068)
best ever =  tensor(0.7166)


In [ ]:
num_weights=16
counter=3000

initial_probabs=[[0.5,0.5] for i in range(num_weights)]
# print("for each wts initial probabs=",initial_probabs)
delta=0.01
options=[1,-1]
for i in range(num_weights):
#     print(initial_probabs[i])
#     print(len(initial_probabs[i]))
    changes=torch.tensor(random.choices(options,weights=initial_probabs[i],k=16))
    changes=changes*delta
changes=torch.FloatTensor(changes).reshape(initial_weights.shape)    
new_weights=initial_weights+changes

for count in range(counter):
    outs=torch.matmul(new_weights,X_train.T)
    outs=outs.T
    acc=accuracy(outs, y_train)
    
    for i in range(num_weights):        
        changes=torch.tensor(random.choices(options,weights=initial_probabs[i],k=16))
        changes=changes*delta
    changes=torch.FloatTensor(changes).reshape(initial_weights.shape)    
    new_weights=new_weights+changes
    if count%100==0:
        print(count,"Accuracy is ",acc)        
#         print(new_weights)
#     break
    

### add some method

In [ ]:
def move_in_same_direction(initial_probabs,epsilon_prob,changes):
    for i in range(len(initial_probabs)):
        if initial_probabs[i][0]>initial_probabs[i][1]:
            initial_probabs[i][0]+=epsilon_prob
            initial_probabs[i][1]-=epsilon_prob            
        else:
            initial_probabs[i][0]-=epsilon_prob
            initial_probabs[i][1]+=epsilon_prob    
    return initial_probabs            
            
def move_in_opp_direction(initial_probabs,epsilon_prob):
    for i in range(len(initial_probabs)):
        if initial_probabs[i][0]>initial_probabs[i][1]:
            initial_probabs[i][0]-=epsilon_prob
            initial_probabs[i][1]+=epsilon_prob            
        else:
            initial_probabs[i][0]+=epsilon_prob
            initial_probabs[i][1]-=epsilon_prob            
    return initial_probabs
        

In [ ]:
initial_weights=torch.FloatTensor([[ 0.0435, -0.2281,  0.2225, -0.1205, -0.1896,  0.2002,  0.2506, -0.2820],
        [-0.2208,  0.3048, -0.2900, -0.1304,  0.3128, -0.1742,  0.1689,  0.0285]])


outs=torch.matmul(initial_weights,X_train.T)
outs=outs.T
initial_acc=accuracy(outs, y_train)

print("Initial acc is ",initial_acc)

In [ ]:
num_weights=16





epsilon_prob=0.01
delta=0.001
epochs=100000


initial_weights=torch.FloatTensor([[ 0.0435, -0.2281,  0.2225, -0.1205, -0.1896,  0.2002,  0.2506, -0.2820],
        [-0.2208,  0.3048, -0.2900, -0.1304,  0.3128, -0.1742,  0.1689,  0.0285]])

print("initial_weights=",initial_weights)
outs=torch.matmul(initial_weights,X_train.T)
outs=outs.T
initial_acc=accuracy(outs, y_train)

print("Initial acc is ",initial_acc)

options=[1,0,-1]
initial_probabs=[[0.33,0.34,0.33] for i in range(num_weights)]
# for i in range(num_weights):
#     num=random.uniform(0.1,1)
#     initial_probabs.append([num,1-num])

print("for each wts initial probabs=",initial_probabs)

# epsilon_prob=random.uniform(0.01,np.min(np.array(initial_probabs)))
# print("epsilon_prob=",epsilon_prob)
changes=[]
for i in range(num_weights):
    choice=random.choices(options,weights=initial_probabs[i],k=1)[0]
    changes.append(choice)    
    
changes=torch.FloatTensor(changes).reshape(initial_weights.shape)    
print("changes",changes)
delta_changes=changes*delta
print("delta_changes",delta_changes)
new_weights=initial_weights+delta_changes
print("Changed wt = ",new_weights)
print(epsilon_prob,delta,epochs,"Running all epochs here")
best_acc=-1
best_acc_weights=None
best_epoch=0
history=[]
for count in range(epochs):
#     #calc training acc
    outs=torch.matmul(new_weights,X_train.T)
    outs=outs.T
    acc=accuracy(outs, y_train)

#     #calc val acc
    outs_val=torch.matmul(new_weights,X_test.T)
    outs_val=outs_val.T
    acc_val=accuracy(outs_val, y_test)
    if best_acc<acc_val:
        best_acc=acc_val
        best_acc_weights=new_weights
        best_epoch=count

    if count%1000==0:                
        print(epsilon_prob,delta,"Training Acc #",count,"is ",acc,"Val Acc is ",acc_val)

    if acc>initial_acc:
#         keep going in the same direction
#         print("improved")
#         print("changes",changes)
#         print("initial_probabs",initial_probabs)
        for i in range(len(changes)):
            for j in range(len(changes[i])):
                indx=i*len(changes[i])+j
                if changes[i][j]==1:
                    initial_probabs[indx][0]+=epsilon_prob/3
                    initial_probabs[indx][1]-=epsilon_prob/6
                    initial_probabs[indx][2]-=epsilon_prob/6                    
                elif changes[i][j]==-1:
                    initial_probabs[indx][0]-=epsilon_prob/3
                    initial_probabs[indx][1]+=epsilon_prob/6
                    initial_probabs[indx][2]+=epsilon_prob/6
#                 elif changes[i][j]==0:
#                     initial_probabs[indx][0]+=epsilon_prob/3
#                     initial_probabs[indx][1]+=epsilon_prob/3
#                     initial_probabs[indx][2]+=epsilon_prob/3 
#         initial_probabs=move_in_same_direction(initial_probabs,epsilon_prob,changes)
    elif acc<initial_acc:
#         print("less")
#         print("changes",changes)
#         print("initial_probabs",initial_probabs)
        for i in range(len(changes)):
            for j in range(len(changes[i])):
                indx=i*len(changes[i])+j
                if changes[i][j]==1:
                    initial_probabs[indx][0]-=epsilon_prob/3
                    initial_probabs[indx][1]+=epsilon_prob/6
                    initial_probabs[indx][2]+=epsilon_prob/6
                    changes[i][j]=-1
                elif changes[i][j]==-1:
                    initial_probabs[indx][0]+=epsilon_prob/3
                    initial_probabs[indx][1]-=epsilon_prob/6
                    initial_probabs[indx][2]-=epsilon_prob/6                    
                    changes[i][j]=1
#                 elif changes[i][j]==0:
#                     initial_probabs[indx][0]+=epsilon_prob/3
#                     initial_probabs[indx][1]+=epsilon_prob/3
#                     initial_probabs[indx][2]+=epsilon_prob/3 
                    
#         initial_probabs=move_in_opp_direction(initial_probabs,epsilon_prob)
    elif acc==initial_acc:
#         print("No change")
        for i in range(len(changes)):
            for j in range(len(changes[i])):
                indx=i*len(changes[i])+j
                if changes[i][j]==1:
                    initial_probabs[indx][0]-=epsilon_prob/3
                    initial_probabs[indx][1]+=epsilon_prob/6
                    initial_probabs[indx][2]+=epsilon_prob/6
                    changes[i][j]=-1
                elif changes[i][j]==-1:
                    initial_probabs[indx][0]+=epsilon_prob/3
                    initial_probabs[indx][1]-=epsilon_prob/6
                    initial_probabs[indx][2]-=epsilon_prob/6                    
                    changes[i][j]=1
# #         print("No change in accuracy")
    history.append(acc_val)
#     print("changed probab = ",initial_probabs)
    changes=[]
    for i in range(num_weights):
        choice=random.choices(options,weights=initial_probabs[i],k=1)[0]
        changes.append(choice)           
    changes=torch.FloatTensor(changes).reshape(initial_weights.shape)          
    delta_changes=changes*delta
    new_weights=new_weights+delta_changes
#     new_weights=new_weights+changes
    initial_acc=acc


# #             # end of all epochs
# #             # add to results


print("probabs=",initial_probabs)
# # print("changes=",changes)
print("Best Accuracy at the end ",best_acc)            
print("Best weights",best_acc_weights)
print("Best epoch",best_epoch)
print("Final proba = ",initial_probabs)

plt.plot(history)


In [ ]:
num_weights=16

dic_res={}
dic_res["epsilon_prob"]=[]
dic_res["delta"]=[]
dic_res["epochs"]=[]
dic_res["best_acc"]=[]
dic_res["best_acc_weights"]=[]




epsilon_prob_list=[0.1,0.01,0.001,0.0001,0.00001]
delta_list=[0.1,0.01,0.001,0.0001,0.00001]
epochs=100000

for epsilon_prob in epsilon_prob_list:
    for delta in delta_list:


        initial_weights=torch.FloatTensor([[ 0.0435, -0.2281,  0.2225, -0.1205, -0.1896,  0.2002,  0.2506, -0.2820],
                [-0.2208,  0.3048, -0.2900, -0.1304,  0.3128, -0.1742,  0.1689,  0.0285]])

        print("initial_weights=",initial_weights)
        outs=torch.matmul(initial_weights,X_train.T)
        outs=outs.T
        initial_acc=accuracy(outs, y_train)

        print("Initial acc is ",initial_acc)

        options=[1,0,-1]
        initial_probabs=[[0.33,0.34,0.33] for i in range(num_weights)]
        print("for each wts initial probabs=",initial_probabs)

        # epsilon_prob=random.uniform(0.01,np.min(np.array(initial_probabs)))
        # print("epsilon_prob=",epsilon_prob)
        changes=[]
        for i in range(num_weights):
            choice=random.choices(options,weights=initial_probabs[i],k=1)[0]
            changes.append(choice)    

        changes=torch.FloatTensor(changes).reshape(initial_weights.shape)    
        print("changes",changes)
        delta_changes=changes*delta
        print("delta_changes",delta_changes)
        new_weights=initial_weights+delta_changes
        print("Changed wt = ",new_weights)
        print(epsilon_prob,delta,epochs,"Running all epochs here")
        best_acc=-1
        best_acc_weights=None
        best_epoch=0
        history=[]
        for count in range(epochs):
        #     #calc training acc
            outs=torch.matmul(new_weights,X_train.T)
            outs=outs.T
            acc=accuracy(outs, y_train)

        #     #calc val acc
            outs_val=torch.matmul(new_weights,X_test.T)
            outs_val=outs_val.T
            acc_val=accuracy(outs_val, y_test)
            if best_acc<acc_val:
                best_acc=acc_val
                best_acc_weights=new_weights
                best_epoch=count

#             if count%1000==0:                
#                 print(epsilon_prob,delta,"Training Acc #",count,"is ",acc,"Val Acc is ",acc_val)

            if acc>initial_acc:
        #         keep going in the same direction    
                for i in range(len(changes)):
                    for j in range(len(changes[i])):
                        indx=i*len(changes[i])+j
                        if changes[i][j]==1:
                            initial_probabs[indx][0]+=epsilon_prob/3
                            initial_probabs[indx][1]-=epsilon_prob/6
                            initial_probabs[indx][2]-=epsilon_prob/6                    
                        elif changes[i][j]==-1:
                            initial_probabs[indx][0]-=epsilon_prob/3
                            initial_probabs[indx][1]+=epsilon_prob/6
                            initial_probabs[indx][2]+=epsilon_prob/6
        #                 elif changes[i][j]==0:
            elif acc<initial_acc:
                for i in range(len(changes)):
                    for j in range(len(changes[i])):
                        indx=i*len(changes[i])+j
                        if changes[i][j]==1:
                            initial_probabs[indx][0]-=epsilon_prob/3
                            initial_probabs[indx][1]+=epsilon_prob/6
                            initial_probabs[indx][2]+=epsilon_prob/6
                            changes[i][j]=-1
                        elif changes[i][j]==-1:
                            initial_probabs[indx][0]+=epsilon_prob/3
                            initial_probabs[indx][1]-=epsilon_prob/6
                            initial_probabs[indx][2]-=epsilon_prob/6                    
                            changes[i][j]=1
        #                 elif changes[i][j]==0:    
            elif acc==initial_acc:
                for i in range(len(changes)):
                    for j in range(len(changes[i])):
                        indx=i*len(changes[i])+j
                        if changes[i][j]==1:
                            initial_probabs[indx][0]-=epsilon_prob/3
                            initial_probabs[indx][1]+=epsilon_prob/6
                            initial_probabs[indx][2]+=epsilon_prob/6
                            changes[i][j]=-1
                        elif changes[i][j]==-1:
                            initial_probabs[indx][0]+=epsilon_prob/3
                            initial_probabs[indx][1]-=epsilon_prob/6
                            initial_probabs[indx][2]-=epsilon_prob/6                    
                            changes[i][j]=1
            history.append(acc_val)
            changes=[]
            for i in range(num_weights):
                choice=random.choices(options,weights=initial_probabs[i],k=1)[0]
                changes.append(choice)           
            changes=torch.FloatTensor(changes).reshape(initial_weights.shape)          
            delta_changes=changes*delta
            new_weights=new_weights+delta_changes
            initial_acc=acc
        print("Best Accuracy at the end ",best_acc)             
        print("probabs=",initial_probabs)
        print("changes=",changes)
        dic_res["epsilon_prob"].append(epsilon_prob)
        dic_res["delta"].append(delta)
        dic_res["epochs"].append(best_epoch)
        dic_res["best_acc"].append(best_acc.detach())
        dic_res["best_acc_weights"].append(best_acc_weights)


    #     print("Best weights",best_acc_weights)
    #     print("Best epoch",best_epoch)
    #     print("Final proba = ",initial_probabs)

        plt.plot(history)
        plt.show()
        print("*"*20)


In [ ]:
df=pd.DataFrame(dic_res)
df=df.sort_values(["best_acc"],ascending=False)
df.head()


In [ ]:
df.to_csv("result_sheets/prelim.csv",index=False)

In [ ]:
num_weights=16


dic_res={}
dic_res["epsilon_prob"]=[]
dic_res["delta"]=[]
dic_res["epochs"]=[]
dic_res["best_acc"]=[]
dic_res["best_acc_weights"]=[]



epsilon_prob_list=[0.1,0.01,0.001,0.0001,0.00001,0.000001]

for epsilon_prob in epsilon_prob_list:

    delta_list=[0.1,0.01,0.001,0.0001,0.00001,0.000001]
    for delta in delta_list:
        epochs_list=[100000]
        for epochs in epochs_list:

            initial_weights=torch.FloatTensor([[ 0.0435, -0.2281,  0.2225, -0.1205, -0.1896,  0.2002,  0.2506, -0.2820],
                    [-0.2208,  0.3048, -0.2900, -0.1304,  0.3128, -0.1742,  0.1689,  0.0285]])


            outs=torch.matmul(initial_weights,X_train.T)
            outs=outs.T
            initial_acc=accuracy(outs, y_train)

#             print("Initial acc is ",initial_acc)

            initial_probabs=[[0.5,0.5] for i in range(num_weights)]
            # print("for each wts initial probabs=",initial_probabs)

            options=[1,-1]
            for i in range(num_weights):
            #     print(initial_probabs[i])
            #     print(len(initial_probabs[i]))
                changes=torch.tensor(random.choices(options,weights=initial_probabs[i],k=16))
                changes=changes*delta
            changes=torch.FloatTensor(changes).reshape(initial_weights.shape)    
            new_weights=initial_weights+changes
            print(epsilon_prob,delta,epochs,"Running all epochs here")
            best_acc=-1
            best_acc_weights=None
            best_epoch=0
            for count in range(epochs):
                #calc training acc
                outs=torch.matmul(new_weights,X_train.T)
                outs=outs.T
                acc=accuracy(outs, y_train)

                #calc val acc
                outs_val=torch.matmul(new_weights,X_test.T)
                outs_val=outs_val.T
                acc_val=accuracy(outs_val, y_test)
                if best_acc<acc_val:
                    best_acc=acc_val
                    best_acc_weights=new_weights
                    best_epoch=count
#                 if count%100==0:                
#                     print(epsilon_prob,delta,"Training Acc #",count,"is ",acc,"Val Acc is ",acc_val)

                if acc>initial_acc:
            #         keep going in the same direction
                    initial_probabs=move_in_same_direction(initial_probabs,epsilon_prob)
                else:
                    initial_probabs=move_in_opp_direction(initial_probabs,epsilon_prob)

            #     print("changed probab = ",initial_probabs)
                for i in range(num_weights):     

                    changes=torch.tensor(random.choices(options,weights=initial_probabs[i],k=16))
                    changes=changes*delta
                changes=torch.FloatTensor(changes).reshape(initial_weights.shape)    
                new_weights=new_weights+changes
                initial_acc=acc

            # end of all epochs
            # add to results
            dic_res["epsilon_prob"].append(epsilon_prob)
            dic_res["delta"].append(delta)
            dic_res["epochs"].append(best_epoch)
            dic_res["best_acc"].append(best_acc.detach())
            dic_res["best_acc_weights"].append(best_acc_weights)


    

In [ ]:
df=pd.DataFrame(dic_res)
df=df.sort_values(["best_acc"],ascending=False)
df.head()


In [ ]:
df.to_csv("result_sheets/prelim.csv",index=False)

In [ ]:
df.head()

In [ ]:
num_weights=16



epsilon_prob_list=[0.1]

for epsilon_prob in epsilon_prob_list:

    delta_list=[0.10]
    for delta in delta_list:
        epochs_list=[500]
        for epochs in epochs_list:

            initial_weights=torch.FloatTensor([[ 0.0435, -0.2281,  0.2225, -0.1205, -0.1896,  0.2002,  0.2506, -0.2820],
                    [-0.2208,  0.3048, -0.2900, -0.1304,  0.3128, -0.1742,  0.1689,  0.0285]])


            outs=torch.matmul(initial_weights,X_train.T)
            outs=outs.T
            initial_acc=accuracy(outs, y_train)

#             print("Initial acc is ",initial_acc)

            initial_probabs=[[0.5,0.5] for i in range(num_weights)]
            # print("for each wts initial probabs=",initial_probabs)

            options=[1,-1]
            for i in range(num_weights):
            #     print(initial_probabs[i])
            #     print(len(initial_probabs[i]))
                changes=torch.tensor(random.choices(options,weights=initial_probabs[i],k=16))
                changes=changes*delta
            changes=torch.FloatTensor(changes).reshape(initial_weights.shape)    
            new_weights=initial_weights+changes
            print(epsilon_prob,delta,epochs,"Running all epochs here")
            best_acc=-1
            best_acc_weights=None
            best_epoch=0
            for count in range(epochs):
                #calc training acc
                outs=torch.matmul(new_weights,X_train.T)
                outs=outs.T
                acc=accuracy(outs, y_train)

                #calc val acc
                outs_val=torch.matmul(new_weights,X_test.T)
                outs_val=outs_val.T
                acc_val=accuracy(outs_val, y_test)
                if best_acc<acc_val:
                    best_acc=acc_val
                    best_acc_weights=new_weights
                    best_epoch=count

#                 if count%10000==0:                
#                     print(epsilon_prob,delta,"Training Acc #",count,"is ",acc,"Val Acc is ",acc_val)

                if acc>initial_acc:
            #         keep going in the same direction
                    print("same")
                    initial_probabs=move_in_same_direction(initial_probabs,epsilon_prob)
                else:
                    print("different")
                    initial_probabs=move_in_opp_direction(initial_probabs,epsilon_prob)

            #     print("changed probab = ",initial_probabs)
                for i in range(num_weights):     

                    changes=torch.tensor(random.choices(options,weights=initial_probabs[i],k=16))
                    changes=changes*delta
                changes=torch.FloatTensor(changes).reshape(initial_weights.shape)    
                new_weights=new_weights+changes
                initial_acc=acc

            # end of all epochs
            # add to results

print("Accuracy at the end ",best_acc)            
print("Best weights",best_acc_weights)
print("Best epoch",best_epoch)

print("probabs=",initial_probabs)
print("changes=",changes)



In [ ]:
changes

In [ ]:


def evaluate(model, X_test, y_test):
    """Evaluate the model's performance on the validation set"""
    outputs = [model.validation_step(X_test,y_test)]
#     print("outputs are ",outputs)
    return model.validation_epoch_end(outputs)



def correct(output, target, topk=(1,)):
    """Computes how many correct outputs with respect to targets

    Does NOT compute accuracy but just a raw amount of correct
    outputs given target labels. This is done for each value in
    topk. A value is considered correct if target is in the topk
    highest values of output.
    The values returned are upperbounded by the given batch size

    [description]

    Arguments:
        output {torch.Tensor} -- Output prediction of the model
        target {torch.Tensor} -- Target labels from data

    Keyword Arguments:
        topk {iterable} -- [Iterable of values of k to consider as correct] (default: {(1,)})

    Returns:
        List(int) -- Number of correct values for each topk
    """

    with torch.no_grad():
        maxk = max(topk)
        # Only need to do topk for highest k, reuse for the rest
        _, pred = output.topk(k=maxk, dim=1, largest=True, sorted=True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(torch.tensor(correct_k.item()))
        return res

class PIMANet(nn.Module):
    """Feedfoward neural network with 1 hidden layer"""
    def __init__(self):
        super().__init__()        
        self.fc1 = nn.Linear(8, 2)          


        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
#         x = F.relu(self.fc1(x))
        return x
        
        
    
    def training_step(self, X_train, y_train):

        out = self(X_train)                  # Generate predictions
        loss = F.cross_entropy(out, y_train) # Calculate loss
        return loss
    
    def validation_step(self, X_test, y_test):
        
        out = self(X_test)                    # Generate predictions
        loss = F.cross_entropy(out, y_test)   # Calculate loss
        acc = accuracy(out, y_test)           # Calculate accuracy
        top_1, top_5 = correct(out, y_test,topk=(1,2))
#         print("Batch is ",batch[1].shape)
        
        top_1=top_1/X_test.shape[0]
        top_5=top_5/X_test.shape[0]

#         print("corr",top_1,top_5)
#         return {'val_loss': loss, 'val_acc': acc}
        return {'val_loss': loss, 'val_acc': acc, 'top_1': top_1, 'top_5': top_5}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        
        batch_top_1s = [x['top_1'] for x in outputs]
#         print(batch_top_1s)
        epoch_top_1 = torch.stack(batch_top_1s).mean()      # Combine top_1
        
        batch_top_5s = [x['top_5'] for x in outputs]
        epoch_top_5 = torch.stack(batch_top_5s).mean()      # Combine top_5
        
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item(),
               'val_top_1': epoch_top_1.item(), 'val_top_5': epoch_top_5.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}, val_top_1: {:.4f}, val_top_5: {:.4f}".format(
                                epoch, result['val_loss'], result['val_acc'], 
                                result['val_top_1'], result['val_top_5']))
        
        

        
        


In [ ]:
def fit(epochs, lr, model, X_train, y_train, X_test, y_test, opt_func=torch.optim.SGD,
               model_state_path=None):
    print("Op tfn is ",opt_func)
    """Train the model using gradient descent"""
    print("At train")
    history = []
    best_so_far=-999    
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        loss = model.training_step(X_train,y_train)
#         print(epoch,loss)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Validation phase
        result = evaluate(model, X_test, y_test)
        if best_so_far<result["val_top_1"]:
            best_so_far=result["val_top_1"]
            if model_state_path:
                torch.save(model.state_dict(), model_state_path)
        
        model.epoch_end(epoch, result)
        history.append(result)

    return history


In [ ]:
torch.manual_seed(20)
model = PIMANet()
model.parameters
if torch.cuda.is_available():
    model=model.cuda()

    
    

print(summary(model, input_size=(1, 8), batch_size=-1))
for nm,params in model.named_parameters():
    print(nm)
    print(params.data)
    
evaluate(model, X_test, y_test)        


epochs=100
lr=0.0001
model_state_path="model_state/mod_CNN.pt"
history2 = fit(epochs, lr, model, X_train, y_train, X_test, y_test , model_state_path=model_state_path)


In [ ]:

if torch.cuda.is_available():
    model.load_state_dict(torch.load(model_state_path))
else:
    model.load_state_dict(torch.load(model_state_path,map_location=torch.device('cpu')))


In [ ]:
res = evaluate(model, X_test, y_test)
print("Best Result after training is ",res)


In [ ]:
for nm,params in model.named_parameters():
    print(nm)
    print(params.data)


In [ ]:
print(X_train[0],"\n",y_train[0])
val=X_train[0:1]
out=model(val)
print("out = ",out)
loss = F.cross_entropy(out, y_train[0:1]) 
print("loss=",loss)

In [ ]:
print(X_train[1],"\n",y_train[1])
val=X_train[1:2]
out=model(val)
print("out = ",out)
loss = F.cross_entropy(out, y_train[1:2]) 
print("loss=",loss)